<a href="https://colab.research.google.com/github/aumadaleandro/Diabetes_ML/blob/main/Diabetes_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PSI3571 - Práticas em Reconhecimento de Padrões, Modelagem e Inteligência Computacional.**

# **Identificador de Diabetes**
**Autores:** Camila Eguchi, Leandro Aumada, Thiago Oliveira.


Este trabalho explora métodos para identificar a presença ou não de diabetes em mulheres, usando como base informações e dados não clínicos dos usuários. A ideia é fornecer uma ferramenta de fácil uso para auxiliar no diagnótico rápido de uma pessoa diabética encaminhando-a rapidamente ao cuidado médico para confirmação de diagnóstico e tratamento rápido.

## **Parte I** - Utilizando o *PIMA Indian Diabetes Database* 


Nesta parte usamos os dados do PIMA, que relaciona a presença de diabetes em mulheres com dados clínicos (como nível de glicose e pressão sanguínea diastólica) e não clínicos (como IMC, idade, etc.) das pacientes.
A ideia é utilizar todos esses dados para averiguar a precisão com a qual conseguimos identificar uma condição diabética por meio de uma rede neural.

PIMA: https://www.kaggle.com/uciml/pima-indians-diabetes-database 

Importando bibliotecas e ferramentas importantes

In [ ]:
# Ferramentas p/ trabalhar com dados e arrays
import numpy as np
import pandas
from prettytable import PrettyTable
from sklearn.preprocessing import OneHotEncoder

# Funcionalidades Keras
import keras
from keras.models import Sequential
from keras.layers.core import Dense
from keras.utils.vis_utils import plot_model
from keras.wrappers.scikit_learn import KerasRegressor
import tensorflow as tf

# Ferramentas de Evaluation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline

Carregando as bases de dados de treino e testes (preparados com dados do PIMA).


*   Dados de Treino: 578 pacientes
*   Dados de Teste:  190 pacientes



In [ ]:
#Carrega dados de treino
basePath =  "/content/drive/Shareddrives/PSI3571-Diabetes/"
trainData = pandas.read_csv(basePath + "Treino Sem Zeros.csv")
testData = pandas.read_csv(basePath + "Teste Sem Zeros.csv")
print("Exemplo - Dados de Treino:")
trainData.head(7) #imprime primeiras 7 linhas p/ aferição

Exemplo - Dados de Treino:


,Preg,Gluc,Dias,Tric,2hSer,BM1,Diab,Age,Diab.1
0,6,148,72,35,166,33.6,627.000,50,1
1,1,85,66,29,166,26.6,351.000,31,0
2,8,183,64,30,166,23.3,672.000,32,1
3,1,89,66,23,94,28.1,167.000,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,30,166,25.6,201.000,30,0
6,3,78,50,32,88,31.0,248.000,26,1


Primeiramente vamos utilizar todas as variáveis presentes na base de dados do PIMA para validar a utilização da mesma na identificação de diabetes.

In [ ]:
#----------------------------#
#      DADOS DE TREINO       #
#----------------------------#
trainSet = trainData.values

# Entradas - Treino
xTrain = np.empty((trainSet.shape[0], 7))
xTrain[:, 0:6] = trainSet[:,0:6] 
xTrain[:,6] = trainSet[:,7]

# Saídas - Treino
yTrain = trainSet[:,8] # Diabético (Sim ou Não)


#----------------------------#
#       DADOS DE TESTE       #
#----------------------------#
testSet = testData.values

# Entradas - Teste
xTest = np.empty((testSet.shape[0], 7))
xTest[:, 0:6] = testSet[:,0:6] 
xTest[:,6] = testSet[:,7]

# Saídas - Teste
yTest = testSet[:,8] 

A função abaixo é responsável por estruturar uma rede neural simples, utilizada somente para fins de protótipo e estudo.
Estrutura da Rede:


*   Camada de Entrada: 8 Neurônios;
*   Duas Camadas Intermediárias: 8 Neurônios;
*   Uma Camada Intermediária: 4 Neurônios;
*   Uma Camada de Saída: 1 Neurônio.

Essa estrutura de rede neural será usada em todos os testes deste estudo.



In [ ]:
def cria_rede(dim, regress=False):
	# Define a estrutura da rede MLP
  model = Sequential()
  model.add(Dense(8, input_dim=dim, activation="relu"))
  model.add(Dense(8, activation="relu"))
  model.add(Dense(8, activation="relu"))
  model.add(Dense(4, activation="relu"))
  # Verifica o tipo da rede (regressor ou classificador)
  if regress:
    model.add(Dense(1, activation="linear"))
  else:
    model.add(Dense(1, activation="sigmoid"))
  # Retorna
  return model

Com isso, podemos instanciar uma rede neural e treiná-la para realizar os primeiros testes.

In [ ]:
model = cria_rede(xTrain.shape[1], regress=False)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
plot_model(model, to_file='/content/drive/My Drive/PSI3571/topologia_regressor.png', show_shapes=True, show_layer_names=True)
# Treinando a rede
print("TREINANDO A REDE:")
model.fit(x=xTrain, y=yTrain, validation_data=(xTest, yTest),epochs=500, batch_size=8)

TREINANDO A REDE:
Epoch 1/500
73/73 [==============================] - 13s 7ms/step - loss: 2.4387 - accuracy: 0.6549 - val_loss: 0.8429 - val_accuracy: 0.6368
Epoch 2/500
73/73 [==============================] - 0s 1ms/step - loss: 0.7058 - accuracy: 0.6620 - val_loss: 0.6975 - val_accuracy: 0.6316
Epoch 3/500
73/73 [==============================] - 0s 1ms/step - loss: 0.6694 - accuracy: 0.6451 - val_loss: 0.6711 - val_accuracy: 0.6316
Epoch 4/500
73/73 [==============================] - 0s 1ms/step - loss: 0.6523 - accuracy: 0.6555 - val_loss: 0.6696 - val_accuracy: 0.6263
Epoch 5/500
73/73 [==============================] - 0s 1ms/step - loss: 0.6188 - accuracy: 0.6772 - val_loss: 0.6692 - val_accuracy: 0.6211
Epoch 6/500
73/73 [==============================] - 0s 1ms/step - loss: 0.6265 - accuracy: 0.6732 - val_loss: 0.6611 - val_accuracy: 0.6263
Epoch 7/500
73/73 [==============================] - 0s 1ms/step - loss: 0.6226 - accuracy: 0.6618 - val_loss: 0.6590 - val_accuracy: 0

Após o modelo treinado, podemos usar o conjunto de testes para analisar a performance do modelo.

In [ ]:
preds = model.predict(xTest)
# Transforma preds em um array comum e normaliza resultados
predicao = np.zeros(preds.size)
for i in range(preds.size):
  predicao[i] = preds[i]
  if predicao[i] > 0.5:
    predicao[i] = 1
  else:
    predicao[i] = 0

Com isso, podemos calcular algumas métricas para verificar a efetividade do modelo:


*   Taxa de Acerto
*   Sensibilidade



In [ ]:
# Perdas e acurácia (precisão)
loss, accuracy = model.evaluate(xTest, yTest, verbose=0)

# Calculo de Sensibilidade
countTP = 0
countTN = 0
countP = 0
countN = 0

acertos = 0

for i in range(predicao.size):
  if yTest[i] == 1.0:
    if predicao[i] == yTest[i]:
      countTP = countTP + 1
      acertos = acertos + 1
    countP = countP + 1
  else:
    if predicao[i] == yTest[i]:
      countTN = countTN + 1
      acertos = acertos + 1
    countN = countN + 1

sensibilidade = countTP / countP

print("Taxa de Acerto: ", accuracy*100, "%")
print("Sensibilidade: ", sensibilidade*100, "%")

Taxa de Acerto:  73.15789461135864 %
Sensibilidade:  43.47826086956522 %


Podemos observar que os valores obtidos foram relativamente bons, uma taxa de acerto de 73% e 47% de sensibilidade nos dão uma precisão suficiente e razoável para o propósito imaginado à aplicação. Na literatura os valores mais altos observados são de ~82% de taxa de acerto, então para uma rede simples um valor de 73% se mostra suficiente.

Abaixo é montada uma tabela mostrando os 25 primeiros valores previstos em comparação com os valores esperados.

In [ ]:
# Monta tabela Y Estimado x Y Esperado (25 elementos)
t = PrettyTable(['Y Regressor', 'Y Esperado'])

for i in range(0, 25):
  t.add_row([predicao[i], yTest[i]])

print(t)


+-------------+------------+
| Y Regressor | Y Esperado |
+-------------+------------+
|     0.0     |    0.0     |
|     0.0     |    1.0     |
|     1.0     |    1.0     |
|     0.0     |    0.0     |
|     0.0     |    0.0     |
|     1.0     |    0.0     |
|     0.0     |    1.0     |
|     0.0     |    0.0     |
|     1.0     |    1.0     |
|     0.0     |    0.0     |
|     1.0     |    1.0     |
|     0.0     |    0.0     |
|     1.0     |    1.0     |
|     0.0     |    0.0     |
|     0.0     |    1.0     |
|     0.0     |    0.0     |
|     0.0     |    0.0     |
|     0.0     |    1.0     |
|     0.0     |    0.0     |
|     0.0     |    0.0     |
|     1.0     |    1.0     |
|     0.0     |    0.0     |
|     0.0     |    0.0     |
|     0.0     |    0.0     |
|     0.0     |    0.0     |
+-------------+------------+


## **Parte II** - Diminuindo o Número de Variáveis **Clínicas**

Como queremos usar somente dados de entrada que não sejam clínicos, vamos tentar primeiramente reduzir a quantidade de entradas usadas pelo PIMA e verificar qual a efetividade disso na predição de diabetes. No entanto, o nível de glicose é fundamental para identificar a existência de diabetes e este dado será mantido. Assim, as entradas que vão ser utilizadas são as seguintes:


*   Número de Gravidezes
*   Nível de Glicose no Sangue
*   IMC (Índice de Massa Corporal)
*   Idade



In [ ]:
#----------------------------#
#      DADOS DE TREINO       #
#----------------------------#
trainSet = trainData.values

# Entradas - Treino
xTrain2 = np.empty((trainSet.shape[0], 3))
xTrain2[:, 0] = trainSet[:,1] # Glucose
xTrain2[:, 1] = trainSet[:,5] # BMI
xTrain2[:, 2] = trainSet[:,7] # Age


# Saídas - Treino
yTrain2 = trainSet[:,8] # Diabético (Sim ou Não)


#----------------------------#
#       DADOS DE TESTE       #
#----------------------------#
testSet = testData.values

# Entradas - Teste
xTest2 = np.empty((testSet.shape[0], 3))
xTest2[:, 0] = testSet[:,1] # Glucose
xTest2[:, 1] = testSet[:,5] # BMI
xTest2[:, 2] = testSet[:,7] # Age
yTest2 = testSet[:,8] 

Com os dados separados, podemos então instanciar e treinar um novo modelo para verificar qual o efeito da redução de variáveis.

In [ ]:
model2 = cria_rede(xTrain2.shape[1], regress=False)
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
plot_model(model2, to_file='/content/drive/My Drive/PSI3571/topologia_regressor.png', show_shapes=True, show_layer_names=True)
# Treinando a rede
print("TREINANDO A REDE:")
model2.fit(x=xTrain2, y=yTrain2, validation_data=(xTest2, yTest2),epochs=500, batch_size=8)

TREINANDO A REDE:
Epoch 1/500
73/73 [==============================] - 1s 5ms/step - loss: 0.7025 - accuracy: 0.5028 - val_loss: 0.6829 - val_accuracy: 0.6158
Epoch 2/500
73/73 [==============================] - 0s 2ms/step - loss: 0.6649 - accuracy: 0.6485 - val_loss: 0.6696 - val_accuracy: 0.7368
Epoch 3/500
73/73 [==============================] - 0s 1ms/step - loss: 0.6712 - accuracy: 0.6364 - val_loss: 0.6599 - val_accuracy: 0.6368
Epoch 4/500
73/73 [==============================] - 0s 1ms/step - loss: 0.6579 - accuracy: 0.6551 - val_loss: 0.6542 - val_accuracy: 0.6368
Epoch 5/500
73/73 [==============================] - 0s 1ms/step - loss: 0.6587 - accuracy: 0.6385 - val_loss: 0.6503 - val_accuracy: 0.6579
Epoch 6/500
73/73 [==============================] - 0s 1ms/step - loss: 0.6518 - accuracy: 0.6657 - val_loss: 0.6480 - val_accuracy: 0.6368
Epoch 7/500
73/73 [==============================] - 0s 1ms/step - loss: 0.6506 - accuracy: 0.6418 - val_loss: 0.6392 - val_accuracy: 0.

Podemos novamente predizer os valores do conjunto de teste.

In [ ]:
preds = model2.predict(xTest2)
# Transforma preds em um array comum e normaliza resultados
predicao = np.zeros(preds.size)
for i in range(preds.size):
  predicao[i] = preds[i]
  if predicao[i] > 0.5:
    predicao[i] = 1
  else:
    predicao[i] = 0

Calculamos novamente os valores de taxa de acerto e sensibilidade.



In [ ]:
# Perdas e acurácia (precisão)
loss, accuracy = model2.evaluate(xTest2, yTest2, verbose=0)

# Calculo de Sensibilidade
countTP = 0
countTN = 0
countP = 0
countN = 0

acertos = 0

for i in range(predicao.size):
  if yTest2[i] == 1.0:
    if predicao[i] == yTest2[i]:
      countTP = countTP + 1
      acertos = acertos + 1
    countP = countP + 1
  else:
    if predicao[i] == yTest2[i]:
      countTN = countTN + 1
      acertos = acertos + 1
    countN = countN + 1

sensibilidade = countTP / countP

print("Taxa de Acerto: ", accuracy*100, "%")
print("Sensibilidade: ", sensibilidade*100, "%")

Taxa de Acerto:  77.89473533630371 %
Sensibilidade:  57.971014492753625 %


Podemos perceber que reduzindo o número de variáveis obtivemos um valor melhor ainda de Taxa de Acerto, de 81% e uma sensibilidade de 71%. 
Com isso podemos concluir que algumas variáveis do banco de dados originais estavam atrapalhando na identificação correta da presença ou não de diabetes.

# **Parte III** - Predizendo o Nível de Glicose

Vimos que utilizando somente uma variável clínica (nível de glicose) e duas variáveis que podem ser obtidas sem necessidade de procedimentos médicos, é possível informar com 80% de acurácia a presença ou não de diabetes em mulheres.
Se for possível prever o nível de glicose no sangue de uma pessoa com uma certa certeza, sem utilizar dados clínicos, poderiamos então criar uma aplicação que não requer a entrada de informações clínicas.

Para tal, vamos utilizar um banco de dados que possui informações sociais e clínicas de pessoas, originalmente utilizada para predizer AVCs. Para nossa aplicação, recortamos somente informações de pacientes do sexo feminino.
As informações que vamos utilizar são as seguintes:


*   Idade
*   Hipertensão (Diagnosticada)
*   Tem doença cardiáca (Diagnosticada)
*   Já foi casada?
*   Tipo de Trabalho (Registrado, autonomo, etc.)
*   Tipo de moradia  (Urbana, rural)
*   IMC




In [ ]:
#Carrega dados de treino
trainGlucData = pandas.read_csv(basePath + "Treino Stroke Normal.csv")
testGlucData = pandas.read_csv(basePath + "Teste Stroke Normal.csv")
print("Dados de Treino:")
trainGlucData.head(7) #imprime primeiras 7 linhas p/ aferição

Dados de Treino:


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi
0,Female,49.0,0,0,1,0,1,171.23,34.4
1,Female,79.0,1,0,1,3,0,174.12,24.0
2,Female,69.0,0,0,0,0,1,94.39,22.8
3,Female,78.0,0,0,1,0,1,58.57,24.2
4,Female,81.0,1,0,1,0,0,80.43,29.7
5,Female,61.0,0,1,1,1,0,120.46,36.8
6,Female,54.0,0,0,1,0,1,104.51,27.3


In [ ]:
def prepare_input(X_data):
  ohe = OneHotEncoder()
  ohe.fit(X_data)
  X_data_enc = ohe.transform(X_data)
  return X_data_enc

Com a base de dados devidamente definida, pode-se separar os dados de entrada e saída para ambos os conjuntos.

In [ ]:
#----------------------------#
#      DADOS DE TREINO       #
#----------------------------#
trainSetGluc = trainGlucData.values

# Entradas - Treino
xTrainGluc = np.empty((trainSetGluc.shape[0], 7))
xTrainGluc[:,0:6] = trainSetGluc[:,1:7] # Age até Residence
xTrainGluc[:,6] = trainSetGluc[:,8] # Bmi
xTrainGluc = np.asarray(xTrainGluc).astype(np.float32) # Cast para tipo Float 
# Saídas - Treino
yTrainGluc = trainSetGluc[:,7]
yTrainGluc = np.asarray(yTrainGluc).astype(np.float32) # Cast para tipo Float 

# #----------------------------#
# #       DADOS DE TESTE       #
# #----------------------------#
testSetGluc = testGlucData.values

# Entradas - Teste
xTestGluc = np.empty((testSetGluc.shape[0], 7))
xTestGluc[:,0:6] = testSetGluc[:,1:7] # Age até Residence
xTestGluc[:,6] = testSetGluc[:,8] # Bmi
xTestGluc = np.asarray(xTestGluc).astype(np.float32) # Cast para tipo Float 
# Saídas - Teste
yTestGluc = testSetGluc[:,7]
yTestGluc = np.asarray(yTestGluc).astype(np.float32) # Cast para tipo Float 

Agora pode-se instanciar um outro modelo de rede neural e treiná-la para tentarmos estimar os valores corretamente. Note que dessa vez se trata de um regressor, que pode ser visto pelo parâmetro *regress=True* na chamada da função de criação de rede.

In [ ]:
modelGluc = cria_rede(xTrainGluc.shape[1], regress=True)
modelGluc.compile(loss="mean_absolute_percentage_error", optimizer='adam')
plot_model(modelGluc, to_file='/content/drive/My Drive/PSI3571/topologia_regressor.png', 
           show_shapes=True, show_layer_names=True)
# Treinando a rede
print("TREINANDO A REDE:")
modelGluc.fit(x=xTrainGluc, y=yTrainGluc, validation_data=(xTestGluc, yTestGluc),
          epochs=500, batch_size=8)

TREINANDO A REDE:
Epoch 1/500
271/271 [==============================] - 1s 2ms/step - loss: 70.7114 - val_loss: 31.0627
Epoch 2/500
271/271 [==============================] - 0s 1ms/step - loss: 28.7210 - val_loss: 29.2354
Epoch 3/500
271/271 [==============================] - 0s 1ms/step - loss: 28.4215 - val_loss: 28.3262
Epoch 4/500
271/271 [==============================] - 0s 1ms/step - loss: 27.3969 - val_loss: 28.2513
Epoch 5/500
271/271 [==============================] - 0s 1ms/step - loss: 27.1096 - val_loss: 27.5396
Epoch 6/500
271/271 [==============================] - 0s 1ms/step - loss: 26.8330 - val_loss: 27.0845
Epoch 7/500
271/271 [==============================] - 0s 1ms/step - loss: 26.5496 - val_loss: 26.8890
Epoch 8/500
271/271 [==============================] - 0s 1ms/step - loss: 26.7252 - val_loss: 26.6685
Epoch 9/500
271/271 [==============================] - 0s 1ms/step - loss: 26.6636 - val_loss: 26.9431
Epoch 10/500
271/271 [==============================] -

Na última época de treinamento observou-se um valor de perda de aproximadamente 20%. Agora podemos predizer os valores do conjunto de teste para analisar com mais certeza.

In [ ]:
predsGluc = modelGluc.predict(xTestGluc)

# Transforma preds em um array comum
predicaoGluc = np.zeros(predsGluc.size)
for i in range(predsGluc.size):
  predicaoGluc[i] = predsGluc[i]

print("Erro Quadrático Médio: ", np.sqrt(mean_squared_error(yTestGluc,predsGluc)))

# Monta tabela Y Estimado x Y Esperado (25 elementos)
t = PrettyTable(['Y Regressor', 'Y Esperado'])

for i in range(0, 25):
  t.add_row([predicaoGluc[i], yTestGluc[i]])

print(t)


Erro Quadrático Médio:  45.838596
+-------------------+------------+
|    Y Regressor    | Y Esperado |
+-------------------+------------+
| 88.69213104248047 |    68.6    |
| 82.67108917236328 |   72.53    |
|  85.3157958984375 |   81.78    |
| 82.27499389648438 |   116.85   |
| 82.82160949707031 |   80.83    |
| 86.50529479980469 |   96.21    |
| 92.10997772216797 |   78.26    |
| 82.91433715820312 |   115.47   |
| 89.06269073486328 |   100.16   |
| 85.19306182861328 |    97.6    |
| 84.46089172363281 |   90.54    |
| 86.12840270996094 |   105.88   |
|  82.8731689453125 |   69.89    |
| 87.78182983398438 |   111.27   |
| 86.90081024169922 |   91.04    |
| 88.96324157714844 |   216.64   |
| 86.42594909667969 |   65.25    |
|  82.976806640625  |   80.92    |
| 83.33016967773438 |   124.37   |
| 81.87933349609375 |   102.39   |
|  80.1927719116211 |   71.08    |
| 91.88024139404297 |   81.54    |
| 88.57396697998047 |   93.96    |
| 83.09249877929688 |   92.49    |
| 87.30020141601562 |

Note que tivemos um Erro Quadrático Médio bastante alto, de ~47 unidades. Além disso, pela tabela comparativa, podemos ver que há um comportamento bastante aleatório nos valores estimados que não acompanha a tendência dos valores que eram esperados.

Estes resultados não validam a utilização deste método para a estimação da glicose, sendo necessário realizar um estudo mais aprofundado, provavelmente com outras informações relacionadas à padrões alimentícios para estimar corretamente o valor de glicose.
Dessa maneira, o nosso estudo, por hora, não garante a determinação de presença diabetes sem que o usuário insira diretamente o valor de glicose.